Try to train the MPS multi-qubit unitaries in addition to the extended gates that are added to the circuits. let's start from the beginning and follow these steps:
1. Load the trained MPS
2. Extract multi-qubit unitaries from the MPS
3. Extend ...

In [3]:
cd ..

/home/abhishekabhishek/git/UnsupGenModbyMPS


In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pennylane as qml
from pennylane import numpy as pnp

from MPScumulant import MPS_c
import mps_circuit_helpers as helpers
import mps_circuit
import metrics

%load_ext autoreload

In [5]:
data = np.load('BStest/BSdata.npy')
data_states = data.reshape(-1, 16).astype(np.int8)

In [36]:
%autoreload
m = MPS_c(16, max_bond_dim=2)
m.loadMPS('BS-2-MPS')

# check the properties of the matrices in the MPS
for i in range(len(m.matrices)):
    tn_core = m.matrices[i]
    print(i, tn_core.shape)

0 (1, 2, 2)
1 (2, 2, 2)
2 (2, 2, 2)
3 (2, 2, 2)
4 (2, 2, 2)
5 (2, 2, 2)
6 (2, 2, 2)
7 (2, 2, 2)
8 (2, 2, 2)
9 (2, 2, 2)
10 (2, 2, 2)
11 (2, 2, 2)
12 (2, 2, 2)
13 (2, 2, 2)
14 (2, 2, 2)
15 (2, 2, 1)


In [37]:
m_pad = helpers.pad_mps(m)
# left canonicalize the padded MPS
m_pad.left_cano()

unitary_list = []
tn_cores = m_pad.matrices
for site_idx in range(len(tn_cores)):
    tn_core = tn_cores[site_idx]
    # this step assumes that the core tensor is a left isometry
    u_mat = helpers.isometry_to_unitary(tn_core.reshape(-1, tn_core.shape[2]))
    unitary_list.append(u_mat)

idx, shape of the padded tensor, updated bond dimension
i = 1, (2, 2, 2), 2
i = 2, (2, 2, 2), 2
i = 3, (2, 2, 2), 2
i = 4, (2, 2, 2), 2
i = 5, (2, 2, 2), 2
i = 6, (2, 2, 2), 2
i = 7, (2, 2, 2), 2
i = 8, (2, 2, 2), 2
i = 9, (2, 2, 2), 2
i = 10, (2, 2, 2), 2
i = 11, (2, 2, 2), 2
i = 12, (2, 2, 2), 2
i = 13, (2, 2, 2), 2
i = 14, (2, 2, 2), 2
bond: 0
bond: 1
bond: 2
bond: 3
bond: 4
bond: 5
bond: 6
bond: 7
bond: 8
bond: 9
bond: 10
bond: 11
bond: 12
bond: 13
bond: 14


In [38]:
unitary_list[2].shape

(4, 4)

In [39]:
helpers.is_unitary(unitary_list[2])

True

In [40]:
dev = qml.device('default.qubit', wires=3)

@qml.qnode(dev)
def circuit(mat):
    qml.QubitUnitary(mat, wires=[0, 1, 2])
    return qml.probs()

In [41]:
circuit(unitary_list[1])

ValueError: Input unitary must be of shape (8, 8) or (batch_size, 8, 8) to act on 3 wires.

In [42]:
u_mat = pnp.tensor(unitary_list[1], requires_grad=True)

In [30]:
u_mat

tensor([[ 5.09668874e-49, -2.50767571e-16,  1.00000000e+00,
          2.51213395e-33],
        [ 9.10132301e-01, -1.08826605e-01, -2.72901835e-17,
         -3.99769889e-01],
        [ 7.79256257e-03,  9.69212796e-01,  2.43047139e-16,
         -2.46101265e-01],
        [ 4.14244457e-01,  2.20869479e-01,  5.53869027e-17,
          8.82959910e-01]], requires_grad=True)

In [25]:
circuit(u_mat)

tensor([9.09720133e-155, 1.71273409e-001, 4.52687073e-001,
        2.14737485e-001, 4.29773492e-010, 3.44145051e-002,
        1.20767529e-001, 6.11999746e-003], requires_grad=True)

In [26]:
qml.grad(circuit)(u_mat)

TypeError: Grad only applies to real scalar-output functions. Try jacobian, elementwise_grad or holomorphic_grad.

In [27]:
jac = qml.jacobian(circuit)(u_mat)
jac

array([[[-1.90758500e-77+0.j, -0.00000000e+00+0.j, -0.00000000e+00+0.j,
         -0.00000000e+00+0.j, -0.00000000e+00+0.j, -0.00000000e+00+0.j,
         -0.00000000e+00+0.j, -0.00000000e+00+0.j],
        [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
          0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
          0.00000000e+00+0.j,  0.00000000e+00+0.j],
        [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
          0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
          0.00000000e+00+0.j,  0.00000000e+00+0.j],
        [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
          0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
          0.00000000e+00+0.j,  0.00000000e+00+0.j],
        [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
          0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
          0.00000000e+00+0.j,  0.00000000e+00+0.j],
        [ 0.00000000

In [28]:
jac.shape

(8, 8, 8)

In [ ]:
help(qml.transforms.decompositions.two_qubit_unitary.two_qubit_decomposition)

In [33]:
decomp = qml.transforms.decompositions.two_qubit_unitary._decomposition_3_cnots(
    u_mat, wires=[0, 1]
)

In [34]:
type(decomp), decomp[0]

(list,
 Rot(tensor(0., requires_grad=True), tensor(0.30680474, requires_grad=True), tensor(3.14159265, requires_grad=True), wires=[0]))

In [35]:
decomp[0].data

[tensor(0., requires_grad=True),
 tensor(0.30680474, requires_grad=True),
 tensor(3.14159265, requires_grad=True)]

In [ ]:
dev = qml.device('default.qubit', wires=3)

@qml.qnode(dev)
def circuit(mat):
    qml.QubitUnitary(mat, wires=[0, 1, 2])
    return qml.probs()